# SDialog dependencies

In [1]:
import os
import json
from tqdm import tqdm

In [2]:
import sdialog
from sdialog import Dialog

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load dialog

In [3]:
original_dialog = Dialog.from_file("dialog_demo.json")
original_dialog.print()

[dialog_id] 1752861530588
[model] client=<botocore.client.BedrockRuntime object at 0x7fc319ec91c0> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' region_name='us-east-1' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')
[seed] 226296126
--- Dialogue Begins ---
[DOCTOR] Hello there. I'm Dr. Smith. Welcome to my office. What brings you in to see me today?
[PATIENT] Good morning, Dr. Smith. I'm John Doe. I've been dealing with a persistent cough and feeling pretty tired for about three weeks now. It's starting to wear me down, and I thought I should get it checked out.
[DOCTOR] I'm sorry to hear you've been feeling unwell, Mr. Doe. A persistent cough and fatigue can certainly be troublesome. Let's get some more details. Can you describe your cough? Is it dry or productive?
[PATIENT] Well, Dr. Smith, it's mostly a dry cough. It's particularly bothersome at night when I'm trying to sleep. You know, it reminds me of the time I was teaching about the Civil War and

# Audio Generation

### Instanciate voices database

In [4]:
from sdialog.audio.voice_database import DummyKokoroVoiceDatabase
dummy_voice_database = DummyKokoroVoiceDatabase()
dummy_voice_database.get_voice(genre="male", age=20)

{'identifier': 'am_echo', 'voice': 'am_echo'}

### Instanciate TTS model

In [5]:
from sdialog.audio.tts_engine import KokoroTTS
tts_engine = KokoroTTS()

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## Setup stage: Audio Dialog and Audio Pipeline

In [6]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

Convert the original dialog into a audio enhanced dialog

In [7]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

## Step 1 : Concatenated utterances

Instanciate the audio pipeline in order to use `Kokoro` (`tts_engine`) as the TTS model and save the outputs of all the dialogs into the directory `./outputs`.

The voices are sampled from the `dummy_voice_database` based on the persona attributes `age` and `gender`, as assigned during the original textual dialog.

In [8]:
os.makedirs("./outputs", exist_ok=True)
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_engine,
    dir_audio="./outputs",
)
# audio_pipeline = AudioPipeline() # Can also be used with default values

Perform the inference of the audio pipeline on the previously converted dialog. In this case we will focus on generating the "unprocessed" audio, which consist of the agregation of all utterances from the dialog. Rather than using the dialog identifier as the name of the directory, we are using here a custom directory name `demo_dialog_kokoro` which will be saved at `./outputs/demo_dialog_kokoro/`. 

In [ ]:
# Generate the audio for the dialog
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    do_step_1=True,
    do_step_2=False,
    do_step_3=False,
    dialog_dir_name="demo_dialog_kokoro",
)

# Path to the audio of the first stage of the audio pipeline
print("Audio generated successfully at:", dialog.audio_step_1_filepath)

[2025-09-29 22:46:57] INFO:root:Dialog audio dir path: ./outputs
[2025-09-29 22:46:57] INFO:root:Audio dialog loaded from the existing file (1752861530588) successfully!
[2025-09-29 22:46:57] INFO:root:Audio dialog saved to the existing file (1752861530588) successfully at the end of the pipeline!


Audio generated successfully at: ./outputs/demo_dialog/exported_audios/audio_pipeline_step1.wav
